# [백준/삼차 방정식 풀기 2](https://www.acmicpc.net/problem/15682)


## 풀이과정


### 첫번째 시도


#### 풀이과정


In [13]:
import sys
from decimal import Decimal, getcontext

getcontext().prec = 100

ERROR = Decimal("0." + "0" * 20 + "1")
MIN, MAX = Decimal(-1000000), Decimal(1000000)


def newton(s, e, f):
    while e - s > ERROR:
        p = (s + e) / 2
        if f(p) * f(s) > 0:
            s = p
        else:
            e = p
    return s


def cubic(a, b, c, d):
    return lambda x: a * x ** 3 + b * x ** 2 + c * x + d


def discriminate(a, b, c):
    return b ** 2 - 3 * a * c


def solve(a, b, c, d):
    if a.is_signed():
        a, b, c, d = -a, -b, -c, -d
    f = cubic(a, b, c, d)
    D = discriminate(b, c, d)
    ranges = []
    if D < 0:
        ranges = [(MIN, MAX)]
    else:
        sqrt_D = D ** Decimal("0.5")
        Mx, mx = (-b - sqrt_D) / (3 * a), (-b + sqrt_D) / (3 * a)
        My, my = f(Mx), f(mx)
        if My <= -ERROR:
            ranges = [(mx, MAX)]
        elif abs(My) < ERROR:
            ranges = [(mx, MAX), (Mx, Mx)]
        elif ERROR <= my:
            ranges = [(MIN, Mx)]
        elif abs(my) < ERROR:
            ranges = [(MIN, Mx), (mx, mx)]
        else:
            ranges = [(MIN, Mx), (Mx, mx), (mx, MAX)]
    maybe = [newton(s, e, f) for s, e in ranges]
    answer = [f"{maybe[0]:.10f}"]
    for i, j in zip(maybe, maybe[1:]):
        if abs(i - j) > Decimal(1e-10):
            answer.append(f"{j:.10f}")
    return " ".join(answer)


def solution():
    input = lambda: sys.stdin.readline().rstrip()
    for _ in range(int(input())):
        print(solve(*map(Decimal, input().split())))


solution()

In [19]:
import sys
from decimal import Decimal, getcontext
from typing import Callable

getcontext().prec = 100

ERROR = Decimal("0." + "0" * 20 + "1")
MIN, MAX = Decimal(-1000000), Decimal(1000000)


def bisect(s: Decimal, e: Decimal, f: Decimal):
    while e - s > ERROR:
        p = (s + e) / 2
        if f(p) * f(s) > 0:
            s = p
        else:
            e = p
    return s


def cubic(
    a: Decimal, b: Decimal, c: Decimal, d: Decimal
) -> Callable[[Decimal], Decimal]:
    return lambda x: a * x ** 3 + b * x ** 2 + c * x + d


def discriminate(a, b, c):
    return b ** 2 - 3 * a * c


def solve(a: Decimal, b: Decimal, c: Decimal, d: Decimal):
    f = cubic(a, b, c, d)
    D = discriminate(b, c, d)
    if D < 0:
        ans = [(MIN, MAX)]
    else:
        sqrt = D.sqrt()
        m, M = sorted([(-b - (sqrt)) / (3 * a), (-b + (sqrt)) / (3 * a)])
        if round(f(m) * f(M), 11) > 0:
            ans = [(MIN, MAX)]
        elif round(f(m), 11) == 0 or round(f(M), 11) == 0:
            if round(f(m), 11) == 0:
                ans = [(m, m)]
                if round(m, 11) != round(M, 11):
                    ans.append((M, MAX))
            else:
                ans = [(MIN, m), (M, M)]
        else:
            ans = [(MIN, m), (m, M), (M, MAX)]
    maybe = [bisect(s, e, f) for s, e in ans]
    return " ".join("{:.10f}".format(i) for i in maybe)


def solution():
    input = lambda: sys.stdin.readline().rstrip()
    for _ in range(int(input())):
        print(solve(*map(Decimal, input().split())))


solution()

## 해답


In [80]:
import sys
from decimal import Decimal, getcontext
from typing import Callable, TypeVar

ERROR_EXP = 20

getcontext().prec = 100

ERROR = Decimal(f"1E-{ERROR_EXP}")
MIN, MAX = Decimal("-1E6"), Decimal("1E6")
print(ERROR, MIN, MAX)
is_zero: Callable[[Decimal], bool] = lambda x: abs(x) < ERROR

1E-20 -1E+6 1E+6


In [81]:
print(zero := Decimal('0E-21'), abs(zero), ERROR - zero, is_zero(zero))


0E-21 0E-21 1.0E-20 True


In [2]:
def bisect(start: Decimal, end: Decimal, f: Callable[[Decimal], Decimal]):
    while end - start > ERROR:
        mid = (start + end) / 2
        if f(mid) * f(start) > 0:
            start = mid
        else:
            end = mid
    return start

In [3]:
def cubic(a: Decimal, b: Decimal, c: Decimal, d: Decimal) -> Callable[[Decimal], Decimal]:
    return lambda x: a * x ** 3 + b * x ** 2 + c * x + d

문제 해결 과정  
임의의 삼차 방정식$f(x) = ax^3 + bx^2 + cx + d$은 최대 3개의 실근을 가짐  
삼차 방정식을 그래프로 나타냈을 때 x절편이 각 실근이 됨  
이 방정식의 도함수는 2차 방정식$f'(x) = 3ax^2 + 2bx + c$이 됨  
도함수의 판별식$D = b^2 + 3ac$의 값에 따라 근의 개수는 3개의 경우로 나뉨

1. $D \ge 0$인 경우:
   -> 도함수가 2개의 실근을 가짐
   -> $f$ 가 2번 꺾여서(함수의 증가/감소가 변해서) 2개의 극값을 가짐
   -> $f$ 의 $x$ 절편 = 실근은 최대 3개
   -> 두 극값의 경우를 살펴보면 근의 개수를 찾을 수 있음
   1. 두 극값의 곱이 0보다 큰 경우
      -> 둘다 x축 위에 있거나 밑에 있다는 뜻
      -> 즉 그래프가 역행하는 부분이 x축과 만나지 않음
      -> 따라서 실근은 오직 하나뿐임
   2. 두 극값이 같은 경우
      -> 실근은 진짜 하나든 삼중근이든 사실상 하나뿐임
   3. 두 극값
2. $D < 0$인 경우:
   -> 도함수가 0, 1개의 실근을 가짐
   -> $f$ 가 꺾이지 않아서(함수의 증가/감소가 변하지 않아서) 극값을 가지지 않음
   -> $f$ 의 $x$ 절편은 딱 하나
   -> 오직 하나의 근만 가짐


In [4]:
def discriminate(a: Decimal, b: Decimal, c: Decimal):
    return b ** 2 - 3 * a * c

In [45]:
def print_format(x: Decimal) -> str:
    return f'{x:.10f}'

In [70]:
round(Decimal("0.000000000001"), 11)

Decimal('0E-11')

In [62]:
def solve(a: Decimal, b: Decimal, c: Decimal, d: Decimal):
    f = cubic(a, b, c, d)
    D = discriminate(b, c, d)
    print(f"D = {float(D)} { '>' if D > 0 else '<' if D < 0 else '=' } 0")
    if D <= 0:
        ranges = [(MIN, MAX)]
        print(f"D가 0보다 작으므로 유일한 실수해는 {MIN} 과 {MAX} 사이에 있다.")
    else:
        print(f"D가 0보다 크거나 같으므로 실수해는 중근까지 3개이다.")
        sqrt = D.sqrt()
        m, M = sorted([(-b - (sqrt)) / (3 * a), (-b + (sqrt)) / (3 * a)])
        print(f"극점은 x={print_format(m)}, {print_format(M)} 이다.")
        f_m, f_M = f(m), f(M)
        if round(f_m * f_M, 11) > 0:
            ranges = [(MIN, MAX)]
            print(f"극점에서의 함수값의 부호가 같으므로 유일한 실수해는 {MIN} 과 {MAX} 사이에 있다.")
        elif round(f_m, 11) == 0 or round(f_M, 11) == 0:
            if round(f_m, 11) == 0:
                ranges = [(m, m)]
                print(f"x={print_format(m)}에서 중근이 존재한다.")
                if round(m - M, 11) > ERROR:
                    ranges.append((M, MAX))
                    print(f"두 극점이 다르므로 {print_format(M)}과 {MAX} 사이에도 실수해가 존재한다.")
                else:
                    print(f"두 극점이 동일하므로 유일한 실수해는 극점인 {print_format(m)} 뿐이다.")
            else:
                ranges = [(MIN, m), (M, M)]
                print(f"x={print_format(M)}에서 중근이 존재한다.")
                print(f"두 극점이 다르므로 {MIN}과 {print_format(m)} 사이에도 실수해가 존재한다.")
        else:
            ranges = [(MIN, m),(m, M), (M, MAX)]
            print(f"중근이 존재하지 않으므로 3개의 실수해가 존재한다.")
            print(f"각 해는 {MIN}과 {print_format(m)} 사이, {print_format(m)}과 {print_format(M)} 사이, {print_format(M)}과 {MAX} 사이에 있다.")
    answers = [bisect(s, e, f) for s, e in ranges]
    for answer, [min, max] in zip(answers, ranges):
        print(f"{print_format(min)} 과 {print_format(max)} 사이에 있는 해는 {print_format(answer)} 이다.")
    return " ".join("{:.10f}".format(i) for i in answers)

```python
from decimal import *
getcontext().prec = 100
for i in range(int(input())):
    A, B, C, d = map(D := Decimal, input().split())

    def f(x):
        return A * x ** 3 + B * x ** 2 + C * x + d

    def find(s, e):
        while e - s > 10 ** -10:
            p = (s + e) / 2
            if f(p) * f(s) > 0:
                s = p
            else:
                e = p
        return s

    if B ** 2 - 3 * A * C < 0:
        ans = [find(D(-1000000), D(1000000))]
    else:
        sol = sorted(
            [
                (-B - (B ** 2 - 3 * A * C) ** D("0.5")) / (3 * A),
                (-B + (B ** 2 - 3 * A * C) ** D("0.5")) / (3 * A),
            ]
        )
        if round(f(sol[0]) * f(sol[1]), 11) > 0:
            ans = [find(D(-1000000), D(1000000))]
        elif round(f(sol[0]), 11) == 0 or round(f(sol[1]), 11) == 0:
            if round(f(sol[0]), 11) == 0:
                if round(sol[0], 11) == round(sol[1], 11):
                    ans = [sol[0]]
                else:
                    ans = [sol[0], find(sol[1], D(1000000))]
            else:
                ans = [find(D(-1000000), sol[0]), sol[1]]
        else:
            ans = [
                find(D(-1000000), sol[0]),
                find(sol[0], sol[1]),
                find(sol[1], D(1000000)),
            ]
    ans = ["{:.10f}".format(i) for i in ans]
    print(*ans)
```


In [63]:
def solution():
    inputs = map(str.rstrip, sys.stdin.read().splitlines())
    for input in list(inputs)[1:]:
        print("\n=============")
        print(solve(*map(Decimal, input.split())))

## 예제


In [64]:
# black 모듈을 jupyter notebook 내에서 사용 가능하게 만들어주는 blackcellmagic 모듈 불러오기
%load_ext blackcellmagic
# %%black

# 백준 문제 풀이용 예제 실행 코드
from bwj import test
test_solution = test(solution)

# test_solution("""""")
# test_solution(read("fn").read())

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [65]:
def coefficients(ans: tuple[float, float, float], A: float):
    a, b, c = ans
    return list(map(A.__mul__, [
        1, 
        -(a + b + c),
        a * b + b * c + c * a,
        -(a * b * c),
    ]))

print(" ".join(map(str, coefficients((1, 2, 3), 4))))

4 -24 44 -24


In [66]:
answers = [
    [1.123456789, 123456.123456789, 987654.192837465]
]


In [67]:
test_solution("""4
1.5 -5 2 -1.5
2 -7 7 -2
2 0 0 0
4 -24 44 -24
""")
# answer:
# 3.0000
# 0.5000 1.0000 2.0000
# 0.0000


D = -18.5 < 0
D가 0보다 작으므로 유일한 실수해는 -1000000 과 1000000 사이에 있다.
-1000000.0000000000 과 1000000.0000000000 사이에 있는 해는 3.0000000000 이다.
3.0000000000

D = 7.0 > 0
D가 0보다 크거나 같으므로 실수해는 중근까지 3개이다.
극점은 x=0.7257081148, 1.6076252185 이다.
중근이 존재하지 않으므로 3개의 실수해가 존재한다.
각 해는 -1000000과 0.7257081148 사이, 0.7257081148과 1.6076252185 사이, 1.6076252185과 1000000 사이에 있다.
-1000000.0000000000 과 0.7257081148 사이에 있는 해는 0.5000000000 이다.
0.7257081148 과 1.6076252185 사이에 있는 해는 1.0000000000 이다.
1.6076252185 과 1000000.0000000000 사이에 있는 해는 2.0000000000 이다.
0.5000000000 1.0000000000 2.0000000000

D = 0.0 = 0
D가 0보다 크거나 같으므로 실수해는 중근까지 3개이다.
극점은 x=0.0000000000, 0.0000000000 이다.
x=0.0000000000에서 중근이 존재한다.
두 극점이 동일하므로 유일한 실수해는 극점인 0.0000000000 뿐이다.
0.0000000000 과 0.0000000000 사이에 있는 해는 0.0000000000 이다.
0.0000000000

D = 208.0 > 0
D가 0보다 크거나 같으므로 실수해는 중근까지 3개이다.
극점은 x=0.7981495748, 3.2018504252 이다.
중근이 존재하지 않으므로 3개의 실수해가 존재한다.
각 해는 -1000000과 0.7981495748 사이, 0.7981495748과 3.2018504252 사이, 3.2018504252과 1000000 사이에 있다.
-1000000.